In [8]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
import nltk
import re

import boto3
from botocore.exceptions import ClientError

In [61]:
class Processing:
        
    def __init__(self):
        self.message = 'This is init'
        
    def datareading(self):
        bucket1 = 's3://hiringtrendanalysis/jobdesc.csv' # already created on S3
        client = boto3.resource('s3')
        df_jobdesc = pd.read_csv(bucket1)
        df_jobdesc = df_jobdesc.drop('Unnamed: 0', axis=1)
        df_jobdesc.dropna(inplace = True)
        
        bucket2 = 's3://hiringtrendanalysis/techwords.csv'
        client = boto3.resource('s3')
        df_keywords = pd.read_csv(bucket2)
#         df_keywords = df_keywords.drop('Unnamed: 0', axis=1)
        
        return df_jobdesc,df_keywords
    
        #tokenize the sentences into words
    
    def tokenizedWords(self,corpus):
#         unigrams = []
#         bigrams = []
        tokenizedWords = []
        
        stop_words = set(stopwords.words('english'))
            
        #Unigrams
        for para in corpus:
            words = nltk.word_tokenize(para)
            for w in words:
                if w not in stop_words and len(w)>1 and not w.isdigit():
                    tokenizedWords.append(w.lower().strip())
                
        #Bigrams
        for para in corpus:
            words = nltk.word_tokenize(para)
            fword= []
            for w in words:
                if w not in stop_words and len(w)>1 and not w.isdigit():
                    fword.append(w)
#             print(fword)
            
            nList = list(ngrams(fword,2))
            
            for w in nList:
#                 print(w)
                tmp = ''.join(w)
                tokenizedWords.append(tmp.lower().strip())

        return tokenizedWords
    
    def datacleansing(self,df_jobdesc,df_keywords ):
        
        jd_tokens = []
        techwords = []
        
        for index, row in df_keywords.iterrows():
            techwords.append(row['words'])
            
#         print(techwords)
        for index, row in df_jobdesc.iterrows():
            corpus = nltk.sent_tokenize(row['job_desc'])
        
            for i in range(len(corpus)):
                corpus [i] = corpus [i].lower()
                corpus [i] = re.sub(r'\W',' ',corpus [i])
                corpus [i] = re.sub(r'\s+',' ',corpus [i])
            
            jd_tk = self.tokenizedWords(corpus)
            
            for i in jd_tk:
                if i in techwords:
#                     print(i)
                    jd_tokens.append(i)
                else:
                    continue
                    
            row['job_desc'] = jd_tokens
            
            jd_tokens = []
        
        return df_jobdesc 

        
    def countWords(self,mydata,df_keywords):
        words_dict = {}
        count = 0
        techwords = []
        
        final = {}
        
        #read final tech words
        for index, row in df_keywords.iterrows():
            techwords.append(row['words'])
        
        #create a dictionary for words in corpus with count
#         words_dict = dict(zip(df_keywords.words, count))
        for index, row in df_keywords.iterrows():
            words_dict.update({row['words']:count})
#             print(words_dict)
        
        temp = words_dict
        
        for index, row in mydata.iterrows():
            for r in row['job_desc']:
                if r in techwords:
#                     print(r,str(words_dict.get(r)))
                    count = words_dict.get(r) + 1
#                     print(count)
                    words_dict.update({r : count})
            
            key = index
            final.update({key:words_dict})
            words_dict = temp
        
        
        return pd.DataFrame.from_dict(final)
#             print(final)
        
        
# class Streaming:
    
        

In [62]:
if __name__ == '__main__':
    
    proc = Processing()
    df_jobdesc,df_keywords = proc.datareading()
    
    mydata = proc.datacleansing(df_jobdesc,df_keywords)
    d = proc.countWords(mydata,df_keywords)
    d.to_csv('test.csv')
    
#     print(a.head())
    
    